In [47]:
import numpy as np
import scipy.io
#from scipy import io
import matplotlib.pyplot as plt
%matplotlib inline
import random
import math
import io
import scipy.optimize, scipy.special
import sklearn.preprocessing
import sklearn.linear_model 
import pandas as pd
from scipy.special import expit
from scipy.sparse import issparse, csr_matrix
from sklearn.linear_model import *
import time

In [2]:
vars = {}
a = scipy.io.loadmat("ex3data1.mat",vars)

In [3]:
X = vars['X']
y = vars['y']
np.place(y,y==10,0)
m = len(X)
theta = np.zeros((X.shape[1]+1,1))
X = np.concatenate((np.ones((m,1)),X),axis=1)

In [4]:
def predict(weights,dataset):
    if(len(weights.shape)==1):
        weights = weights[:,np.newaxis]
    #print("weights.shape=",weights.shape)
    #print("dataset.shape=",dataset.shape)
    prediction = scipy.special.expit(np.dot(dataset,weights))
    return prediction

In [5]:
def cost(weights,dataset,labels,gamma=0):
    if(len(weights.shape)==1):
        weights = weights[:,np.newaxis]
    regularizer = (gamma/(2*m))*np.sum(np.delete(labels,0,0))
    prediction = predict(weights,dataset)
    cost = -1/m * np.sum(np.dot(labels.T,np.log(prediction))+np.dot((1-labels).T,np.log(1-prediction))) +regularizer
    #print(cost)
    return cost

In [6]:
def one_hot_encoding(labels,class_nr):
    new_labels = np.copy(labels)
    np.place(new_labels,new_labels==class_nr,1) 
    np.place(new_labels,new_labels!=1,0)
    return new_labels

In [7]:
def gradient(weights,dataset,labels,gamma=0):
    if(len(weights.shape)==1):
        weights = weights[:,np.newaxis]
    regularizer = (gamma/m) * (np.insert(np.delete(theta,0,0),0,0,0))
    prediction = predict(weights,dataset)
    grad = 1/m*np.dot(X.T,(prediction-labels)) +regularizer
    return np.ndarray.flatten(grad)

In [8]:
def hessian(weights,dataset,labels,gamma=0.1):
    W = predict(weights,dataset) * (1-predict(weights,dataset))
    W = np.diag(W.flatten())
    return np.dot(np.dot(-dataset.T,W),dataset)

In [9]:
hessian(theta,X,one_hot_encoding(y,1)).shape

(401, 401)

In [55]:
def minimize(weights,dataset,labels):
    return scipy.optimize.minimize(cost,weights,method='Nelder-Mead',jac=gradient,args=(dataset,labels),tol=0.0001)

In [41]:
def minimize2(weights,dataset,labels):
    return scipy.optimize.fmin_tnc(cost,weights,fprime=gradient,args=(dataset,labels),xtol=0.0001)

In [31]:
def train(dataset,labels,class_nr):
    new_labels = one_hot_encoding(labels,class_nr)
    theta = np.zeros((dataset.shape[1],1))
    return minimize(weights = theta,dataset=dataset,labels=new_labels)

In [56]:

start = time.time()
print("hello")
for i in range(0,10):
    if i == 0:
        all_theta = train(X,y,i)['x']
        all_theta = all_theta[:,np.newaxis]
    else:
        print(i)
        curr_theta = train(X,y,i)['x']
        curr_theta = curr_theta[:,np.newaxis]
        all_theta = np.concatenate((all_theta,curr_theta),axis=1)
end = time.time()
print(end - start)
   

hello


KeyboardInterrupt: 

In [34]:
p = predict(all_theta,X)

p.shape

(5000, 10)

In [35]:
y_hat = np.argmax(p,axis=1)
y_hat = y_hat[:,np.newaxis]

In [36]:
difference = y - y_hat

In [37]:
errors = 0
kind_of_error = []

In [38]:
for i in range(0,len(y)-1):
    if difference[i] != 0:
        errors +=1
        kind_of_error.append((y_hat[i],y[i]))

In [39]:
(len(y)-errors) / len(y)

0.8788